# Viewing the Data

In [ ]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
plt.ioff()

import blib
blib.useTheme("dark")

import src.radarkit as radarkit
import src.radarkit.chart as chart

# RHI

In [ ]:
files = sorted(glob.glob("data/moment/20231004/*.xz"))
assert len(files) > 0, "No files found"
file = files[0]
print(f"Selected file {file}")

In [ ]:
sweep = radarkit.sweep.Sweep(file)

rhi = chart.ChartRHI()
rhi.set_data(sweep)

In [ ]:
rhi.fig

# PPI

In [ ]:
files = sorted(glob.glob("data/moment-ppc2/20230130/*-Z.nc"))
assert len(files) > 0, "No files found"
file = files[0]
print(f"Selected file {file}")

In [ ]:
sweep = radarkit.sweep.Sweep(file)

ppi = chart.ChartPPI()
ppi.set_data(sweep)

In [ ]:
ppi.fig

In [ ]:
plt.close()